In [ ]:
from cracknuts import CrackerG1

In [ ]:
g1 = CrackerG1('192.168.0.21')
g1.connect(force_update_bin=True, force_write_default_config=True)

In [ ]:
import cracknuts as cn

In [ ]:
import random
import time
from cracknuts.cracker import serial


cmd_set_aes_enc_key = "01 00 00 00 00 00 00 10"
cmd_aes_enc = "01 02 00 00 00 00 00 10"

aes_key = "11 22 33 44 55 66 77 88 99 00 aa bb cc dd ee ff"
aes_data_len = 16

sample_length = 20000

def init(c: CrackerG1):
    c.nut_voltage_enable()
    c.nut_voltage(3.3)
    c.nut_clock_enable()
    c.nut_clock_freq('8M')
    c.uart_io_enable()
    c.osc_sample_clock('48m')
    c.osc_sample_length(sample_length)
    c.osc_trigger_source('N')
    c.osc_analog_gain('B', 10)
    c.osc_trigger_level(0)
    c.osc_trigger_mode('E')
    c.osc_trigger_edge('U')
    c.uart_config(baudrate=serial.Baudrate.BAUDRATE_115200, bytesize=serial.Bytesize.EIGHTBITS, parity=serial.Parity.PARITY_NONE, stopbits=serial.Stopbits.STOPBITS_ONE)

    time.sleep(2)
    cmd = cmd_set_aes_enc_key + aes_key
    status, ret = c.uart_transmit_receive(cmd, timeout=1000, rx_count=6)

def do(c: CrackerG1, count: int):
    plaintext_data = random.randbytes(aes_data_len)
    tx_data = bytes.fromhex(cmd_aes_enc.replace(' ', '')) + plaintext_data
    status, ret = c.uart_transmit_receive(tx_data, rx_count= 6 + aes_data_len, is_trigger=True)
    
    return {
        "plaintext": plaintext_data,
        "ciphertext": ret[-aes_data_len:],
        "key": bytes.fromhex(aes_key)
    }


def finish(c):
    ...
    # print('optional behavior')


acq = cn.new_acquisition(g1, init=init, do=do, finish=finish)

In [ ]:
cn.panel(acq)

In [ ]:
from cracknuts.cracker import CrackerS1

CrackerS1.nut_voltage(g1, 3.5)
g1.nut_voltage_enable()

In [ ]:
g1.nut_voltage(3.5)
g1.nut_voltage_enable()

In [ ]:
g1.get_current_config()

In [ ]:
g1.glitch_vcc_arm()
g1.glitch_vcc_normal(3.5)